First Laboratory work

Vadim Čeremisinov, 4 grupė, 1 pogrupis
LSP 2110545

Classes - Duck, Banana, Sea lion


In [15]:
import torch
from torchvision import datasets
from torchvision.transforms import transforms
import numpy as np
from torchvision import models

Setting classes, creating folders for photos from openimages:

In [16]:
%pip install openimages
import os
from openimages.download import download_dataset

data_dir = "data"
number_for_samples = 340
classes = ["Lemon", "Banana", "Sea lion"]   #Lemon, Orange

if os.path.exists(data_dir):
  pass
elif not os.path.exists(data_dir):
  os.makedirs(data_dir)

Downloading photos of chosen classes from openimages with download_dataset:

In [ ]:
download_dataset(data_dir, classes, limit=number_for_samples)

Transforming/standartizing downloaded photos:

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
std=[0.229, 0.224, 0.225])

all_transforms = transforms.Compose([
transforms.Resize((224, 224)),
transforms.ToTensor(),
normalize,
])

Creating dataset:

In [ ]:
dataset = datasets.ImageFolder(root='data', transform=all_transforms)

Loading photos batch into batch_loader:

In [ ]:
batch_loader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=2)

GPU sanity check:

In [ ]:
torch.cuda.is_available()

Initializing neural network model:

In [ ]:
model = models.vgg16(pretrained = True)
model.to("cuda")
model.eval()

Metrics calculation function:



In [ ]:
def calculate_metrics(ground_truth, predictions, threshold = 0.9):
  predictions_thresholded = (predictions > threshold).astype(np.float64)
  TP = np.sum(np.bitwise_and(ground_truth == 1, predictions_thresholded == 1))
  TN = np.sum(np.bitwise_and(ground_truth == 0, predictions_thresholded == 0))
  FP = np.sum(np.bitwise_and(ground_truth == 0, predictions_thresholded == 1))
  FN = np.sum(np.bitwise_and(ground_truth == 1, predictions_thresholded == 0))
  metrics = {}
  metrics['accuracy'] = (TP + TN) / (TP + FP + TN + FN)
  metrics['recall'] = TP / (TP + FN)
  metrics['precision'] = TP / (TP + FP)
  metrics['f1'] = 2 * (metrics['precision'] * metrics['recall']) / (metrics['precision'] + metrics['recall'])
  return metrics

Additional functions for probabilities counting:

In [ ]:
def calculate_prob(arr2D, class_id, label_arr, class_label):
  result = []
  label_bool = 1.0
  for i in range(0, len(label_arr)):
    if class_label == label_arr[i]:
      label_bool = 1.0
    else:
      label_bool = 0.0
    result.append((arr2D[i][class_id], label_bool))
  return result

def extract_prob(arr):
  result = []
  for prob, truth in arr:
    result.append(prob)
  return result

def extract_gt(arr):
  result = []
  for prob, truth in arr:
    result.append(truth)
  return result

def extractAndCalculate(gt_prob_list, threshold):
  gt_list = extract_gt(gt_prob_list)
  pred_list = extract_prob(gt_prob_list)
  return calculate_metrics(np.array(gt_list), np.array(pred_list), threshold)


Setting threshold value:


In [ ]:
threshold = 0.95

Counting accuracy, recall, precision and F1 metrics:



In [ ]:
probabilities_list = []
labels_list = []

lemon_class_id = 951
banana_class_id = 954
sea_lion_class_id = 150


duck_truth_probablity_list = []
banana_truth_probablity_list = []
sea_lion_truth_probablity_list = []

for images, labels in batch_loader:
  images, labels = images.cuda(), labels.cuda()
  outputs = model(images)
  probabilities = torch.sigmoid(outputs).detach().cpu().numpy()
  labels_array = labels.cpu().numpy()

  duck_truth_probablity_list.extend(calculate_prob(probabilities, lemon_class_id, labels_array, 0))
  banana_truth_probablity_list.extend(calculate_prob(probabilities, banana_class_id, labels_array, 1))
  sea_lion_truth_probablity_list.extend(calculate_prob(probabilities, sea_lion_class_id, labels_array, 2))


print(f"Duck statistics: {extractAndCalculate(duck_truth_probablity_list, threshold)}")
print(f"Banana statistics: {extractAndCalculate(banana_truth_probablity_list, threshold)}")
print(f"Sea lion statistics: {extractAndCalculate(sea_lion_truth_probablity_list, threshold)}")
